In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DP, DP_carb, DP_both
from Logic import logic_bat
from Funcs_Logic_DP import action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from MPC import MPC, MPC_carb, MPC_both, MPCModel
from Predictions import RF

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
carb = load_series("e",None,"carb_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,8.7,9.7,11.6,9.5,2022,12,31,5,19
2022-12-31 20:00:00,6.9,9.6,7.9,7.9,2022,12,31,5,20
2022-12-31 21:00:00,5.3,7.0,8.2,5.8,2022,12,31,5,21
2022-12-31 22:00:00,5.6,6.7,8.0,6.3,2022,12,31,5,22


In [3]:
merged = merge(house)
merged

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


In [4]:
bat_capacity=13
stepsize=2
Start="2022-06-19 00:00:00"

using_N=False

if using_N:
    N = 24*7
    End = pd.date_range(start=Start, periods=N, freq="h")[-1]

else:
    End = "2022-06-19 23:00:00"
    N=len(pd.date_range(start=Start,end=End,freq="h"))
    
remainder = N%24

if remainder==0:
    print(f"Period is of size {N} hours = {N//24} days\nPeroid from {Start} to {End}")
elif remainder==1:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hour\nPeroid from {Start} to {End}")
else:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hours\nPeroid from {Start} to {End}")

Period is of size 24 hours = 1 days
Peroid from 2022-06-19 00:00:00 to 2022-06-19 23:00:00


## Running all models

In [ ]:
battery_no   = Battery(max_capacity=bat_capacity,max_charge=0)
battery      = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat_price  = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat_carb   = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat_both   = Battery(max_capacity=bat_capacity,max_charge=7)
MPCbat_price = Battery(max_capacity=bat_capacity,max_charge=7)
MPCbat_carb  = Battery(max_capacity=bat_capacity,max_charge=7)
MPCbat_both  = Battery(max_capacity=bat_capacity,max_charge=7)

rf = RF(house)

Mpc = MPCModel(house, 0.1, 0.0, 1, MPCbat_price.max_charge, MPCbat_price.max_capacity)
ratio=0.5


series_no_battery  = pd.DataFrame(columns=merged.columns)
series_battery     = pd.DataFrame(columns=merged.columns)
series_battery_DP_price  = pd.DataFrame(columns=merged.columns)
series_battery_DP_carb   = pd.DataFrame(columns=merged.columns)
series_battery_DP_both   = pd.DataFrame(columns=merged.columns)
series_battery_MPC_price = pd.DataFrame(columns=merged.columns)
series_battery_MPC_carb  = pd.DataFrame(columns=merged.columns)
series_battery_MPC_both  = pd.DataFrame(columns=merged.columns)

preds = pd.DataFrame() 

Start_i = Start

num_loops = int(np.ceil(N/stepsize))
remainder = N%stepsize
length = stepsize
for i in range(num_loops):
    if i == num_loops-1:
        length = length if remainder == 0 else remainder
            
    End_i = pd.date_range(start=Start_i,periods=24,freq="h")[-1]
    merged_i = merged.loc[Start_i:End_i]
    print(f"Loop number {i+1} of {num_loops}")
    print(f"Predicting period from {Start_i} to {End_i}")
    
    
    pred_i = rf.get_predictions(Start_i, End_i)
    
    
    print(f"Taking actions the first {length} actions from {Start_i} to {pred_i.index[length-1]}")
    
    temp_no_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery_no), logic_bat)
    temp_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery), logic_bat)
    
    print("\rRunning DP price optimization",end="")
    actions_DP_price = DP(Start_i,End_i,house,pred_i,deepcopy(DPbat_price),byday=True,ints=True,degrade=False,verbose=False)
    
    print("\r                                             ",end="")
    print("\rRunning DP carbon emissions optimization",end="")
    actions_DP_carb  = DP_carb(Start_i,End_i,house,pred_i,deepcopy(DPbat_carb),byday=True,ints=True,degrade=False,verbose=False)
    
    print("\r                                             ",end="")
    print("\rRunning DP price and emissions optimization",end="")
    actions_DP_both  = DP_both(Start_i,End_i,house,pred_i,deepcopy(DPbat_both),byday=True,ints=True,degrade=False,verbose=False,ratio=ratio)
    
    print("\r                                             ",end="")
    print("\rRunning MPC price optimization",end="")
    actions_MPC_price = MPC(Start_i,End_i,pred_i,deepcopy(MPCbat_price),Mpc,byday=True,verbose=False)
    
    print("\r                                             ",end="")
    print("\rRunning MPC carbon emissions optimization",end="")
    actions_MPC_carb  = MPC_carb(Start_i,End_i,pred_i,deepcopy(MPCbat_carb),Mpc,byday=True,verbose=False)
    
    print("\r                                             ",end="")
    print("\rRunning MPC price and emissions optimization",end="")
    actions_MPC_both = MPC_both(Start_i,End_i,pred_i,deepcopy(MPCbat_both),Mpc,ratio=ratio,byday=True,verbose=False)
    print("\r                                             ",end="")
    print("\rDone!",end="")
    
    series_no_battery_i         = action_rollout(merged_i.iloc[:length], battery_no, temp_no_bat[:length])
    series_battery_i            = action_rollout(merged_i.iloc[:length], battery, temp_bat[:length])
    series_battery_DP_price_i   = action_rollout(merged_i.iloc[:length], DPbat_price, actions_DP_price)
    series_battery_DP_carb_i    = action_rollout(merged_i.iloc[:length], DPbat_carb, actions_DP_carb)
    series_battery_DP_both_i    = action_rollout(merged_i.iloc[:length], DPbat_both, actions_DP_both)
    series_battery_MPC_price_i  = action_rollout(merged_i.iloc[:length], MPCbat_price, actions_MPC_price)
    series_battery_MPC_carb_i   = action_rollout(merged_i.iloc[:length], MPCbat_carb, actions_MPC_carb)
    series_battery_MPC_both_i   = action_rollout(merged_i.iloc[:length], MPCbat_both, actions_MPC_both)
    
    
    Start_i = pd.date_range(start=Start_i,periods=length+1,freq="h")[-1]
    
    preds=pd.concat([preds,pred_i[:length]])
    series_no_battery  = pd.concat([series_no_battery,series_no_battery_i])
    series_battery     = pd.concat([series_battery,series_battery_i])
    series_battery_DP_price  = pd.concat([series_battery_DP_price,series_battery_DP_price_i])
    series_battery_DP_carb   = pd.concat([series_battery_DP_carb,series_battery_DP_carb_i])
    series_battery_DP_both   = pd.concat([series_battery_DP_both,series_battery_DP_both_i])
    series_battery_MPC_price  = pd.concat([series_battery_MPC_price,series_battery_MPC_price_i])
    series_battery_MPC_carb   = pd.concat([series_battery_MPC_carb,series_battery_MPC_carb_i])
    series_battery_MPC_both   = pd.concat([series_battery_MPC_both,series_battery_MPC_both_i])
    
    print()
    print()
preds

Loop number 1 of 12
Predicting period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Taking actions the first 2 actions from 2022-06-19 00:00:00 to 2022-06-19 01:00:00
Running MPC price optimization               

In [ ]:
series_no_battery["cost_cummulative"] = series_no_battery["cost"].cumsum(axis=0)
series_battery["cost_cummulative"] = series_battery["cost"].cumsum(axis=0) 
series_battery_DP_price["cost_cummulative"] = series_battery_DP_price["cost"].cumsum(axis=0)
series_battery_DP_carb["cost_cummulative"] = series_battery_DP_carb["cost"].cumsum(axis=0)
series_battery_DP_both["cost_cummulative"] = series_battery_DP_both["cost"].cumsum(axis=0)
series_battery_MPC_price["cost_cummulative"] = series_battery_MPC_price["cost"].cumsum(axis=0)
series_battery_MPC_carb["cost_cummulative"] = series_battery_MPC_carb["cost"].cumsum(axis=0)
series_battery_MPC_both["cost_cummulative"] = series_battery_MPC_both["cost"].cumsum(axis=0)

series_no_battery["emission_cummulative"] = series_no_battery["emission"].cumsum(axis=0)
series_battery["emission_cummulative"] = series_battery["emission"].cumsum(axis=0) 
series_battery_DP_price["emission_cummulative"] = series_battery_DP_price["emission"].cumsum(axis=0)
series_battery_DP_carb["emission_cummulative"] = series_battery_DP_carb["emission"].cumsum(axis=0)
series_battery_DP_both["emission_cummulative"] = series_battery_DP_both["emission"].cumsum(axis=0)
series_battery_MPC_price["emission_cummulative"] = series_battery_MPC_price["emission"].cumsum(axis=0)
series_battery_MPC_carb["emission_cummulative"] = series_battery_MPC_carb["emission"].cumsum(axis=0)
series_battery_MPC_both["emission_cummulative"] = series_battery_MPC_both["emission"].cumsum(axis=0)

## No battery

In [ ]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

## Simple logic

In [ ]:
print_price_summary(series_battery)
logic_series_print(series_battery)

## DP

In [ ]:
print_price_summary(series_battery_DP_price)
logic_series_print(series_battery_DP_price)

In [ ]:
print_price_summary(series_battery_DP_carb)
logic_series_print(series_battery_DP_carb)

In [ ]:
print_price_summary(series_battery_DP_both)
logic_series_print(series_battery_DP_both)

## MPC

In [ ]:
print_price_summary(series_battery_MPC_price)
logic_series_print(series_battery_MPC_price)

In [ ]:
print_price_summary(series_battery_MPC_carb)
logic_series_print(series_battery_MPC_carb)

In [ ]:
print_price_summary(series_battery_MPC_both)
logic_series_print(series_battery_MPC_both)

In [ ]:
#series_no_battery.to_csv("series_no_battery.csv")
#series_battery.to_csv("series_battery.csv")
#series_battery_DP_price.to_csv("series_battery_DP_price.csv")
#series_battery_DP_carb.to_csv("series_battery_DP_carb.csv")